In [30]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import  ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


In [35]:

load_dotenv()
SERVICE_KEY = os.getenv('OPEN_API_KEY')

llm = ChatOpenAI( openai_api_key= SERVICE_KEY,model_name="gpt-3.5-turbo-1106", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    
    # Debugging: Check the type of result
    print("Type of result:", type(result))
    print("Result:", result)

    # Ensure result is a string
    if isinstance(result, dict):
        result_str = str(result)  # Convert dict to a string if needed
    elif not isinstance(result, str):
        result_str = str(result)  # Ensure any other type is converted to string
    else:
        result_str = result

    memory.save_context(
        {"input": question},
        {"output": result_str},  # Ensure it's a string
    )


In [36]:
invoke_chain("My name is sam")

Nice to meet you, Sam! How can I assist you today?Type of result: <class 'langchain.schema.messages.AIMessageChunk'>
Result: content='Nice to meet you, Sam! How can I assist you today?'


In [37]:
invoke_chain("What is my name?")

Your name is Sam.Type of result: <class 'langchain.schema.messages.AIMessageChunk'>
Result: content='Your name is Sam.'


In [38]:
invoke_chain("What is my name?")

Your name is Sam.Type of result: <class 'langchain.schema.messages.AIMessageChunk'>
Result: content='Your name is Sam.'
